In [1]:
import onnxruntime 
onnx_session = onnxruntime.InferenceSession(
    'trfs-model.onnx', )

In [275]:
### PDF
import requests
import io
import pypdfium2 as pdfium
text=[]

r = requests.get(f'https://angelmatch-store.sfo3.digitaloceanspaces.com/pitch-decks/pdf/53.pdf')
filestream = io.BytesIO(r.content)
try:
        text=[]
        pdf = pdfium.PdfDocument(filestream)
        for page in pdf:
            textpage = page.get_textpage().get_text()
            text.append(textpage)
        text = " ".join(" ".join(text).split('\n'))
except Exception as e: print(e)

# text=" ".join(open('/Users/apsys/Downloads/startup-imperial/9cd3882116c9da4b2fdb13938a9a34ec.txt','r+').read().split('\n'))
text

'We believe that entrepreneurs will make the world better for us alL  Ark makes the world better for entrepreneurs.    \x0c  \x0cFinancing from Financing from  institutions with VCs aimed at ancient methods high risk           Enables growth po yooe Dee Uae ciao perme     | Unable bo fund teh risk & baliible |             \x0c  \x0c     \x0c  \x0c  \x0cArk Investor presentation  Ark — The Precision Finance Company     Precise tech plotfonmn Pracise landing model  Tech piotfornn with bow Ged Preven landing model thet has: P| integrations to large scale, found mispriced risk in the  row dolosets enables Al morkek Longe customer pigetinen Powered predictive modelling built in. 2 months bn stealth mode,  ond rak OsSessments.  The Preceion Finance Company        Precise funding platform  ‘Scotobie funding. Signeficant credit focilities secured ot launch, sanobting profitgble loniding from  day 2. \x0cInvestor preaenootion  pAN  |  businesses to graw faster, enables owners to maintain contro

In [297]:
from transformers import AutoTokenizer
import numpy as np
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
inputs = tokenizer(
     "Mid level, senior. Great, but not so much", max_length=512, padding="max_length", truncation=True,
)

feed = dict(
    input_ids=np.array([inputs["input_ids"]]).astype("int64"),
    token_type_ids=np.array([inputs['token_type_ids']]).astype("int64"),
    attention_mask=np.array([inputs["attention_mask"]]).astype("int64"),
)

In [298]:
output = onnx_session.run(None, feed)
(np.tanh(output[0]))

array([[ 0.40036502,  0.1452033 ,  0.04392549,  0.155462  ,  0.7573134 ,
        -0.832991  ,  0.1582331 , -0.12531796, -0.25995803, -0.47400296]],
      dtype=float32)

In [287]:
def softmax(x):
    return(np.exp(x)/np.exp(x).sum())

In [299]:
predictions = np.squeeze(softmax(output[0]))
arglist = np.argsort(np.sign(np.tanh(output[0]).mean())*(predictions))
if np.sign(np.tanh(output[0]).mean())==-1.0:
    arglist=arglist[1:]

In [300]:
arglist

array([0, 6, 3, 1, 2, 7, 8, 9, 5])

In [301]:
labels = ['<100K$', '100-125K$', '125-150K$', '150-375K$', '375-500K$', '500-600K$', '0.6-1.1M$', '1.1-2M$', '2-6.5M$', '6.5M$>']
preds=[labels[x] for x in arglist[0:3]]

In [302]:
preds

['<100K$', '0.6-1.1M$', '150-375K$']